In [ ]:
# ------------------------------------------------------------------------------
#
#   Copyright 2024 Valory AG
#
#   Licensed under the Apache License, Version 2.0 (the "License");
#   you may not use this file except in compliance with the License.
#   You may obtain a copy of the License at
#
#       http://www.apache.org/licenses/LICENSE-2.0
#
#   Unless required by applicable law or agreed to in writing, software
#   distributed under the License is distributed on an "AS IS" BASIS,
#   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#   See the License for the specific language governing permissions and
#   limitations under the License.
#
# ------------------------------------------------------------------------------

"""Script to simulate markets."""

import random
from collections import Counter
from typing import Any, List

import ipympl
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from mpl_toolkits.mplot3d import Axes3D
from omen_subgraph_utils import get_fpmms
from sage.misc.prandom import shuffle as sage_shuffle
from scipy.stats import describe

# Enable matplotlib inline backend
%matplotlib widget


fpmms = get_fpmms("0x89c5cc945dd550BcFfb72Fe42BfF002429F46Fec")


def plot_histogram(values: List[Any], title: str) -> None:
    """Plot histogram and cumulative histogram."""
    summary = describe(values)
    print(summary)

    plt.figure(figsize=(12, 5))
    n_bins = 200
    bin_size = (max(values) - min(values)) / n_bins 
    bins = np.arange(min(values), max(values)+bin_size, bin_size)

    # Plot histogram
    plt.subplot(1, 2, 1)
    plt.hist(values, bins=bins, color='blue', density=True)
    plt.xlabel('Values')
    plt.ylabel('Count')
    plt.title(f'Histogram of {title}')

    # Plot cumulative histogram
    plt.subplot(1, 2, 2)
    plt.hist(values, bins=bins, color='orange', density=True, cumulative=True)
    plt.xlabel('Values')
    plt.ylabel('Cumulative count')
    plt.title(f'Cumulative histogram of {title}')

    plt.tight_layout()
    plt.show()


def fpmm_trade(amount: float, option: int, L0: float, L1: float, fee: float=0.02) -> (int, int, int):
    """Simulates an fpmm trade with liquidity L0 and L1 for options 1 and 2, respectively."""
    k = L0*L1
    amount = amount*(1-fee)
    if option == 0:
        return (L0+amount, (L0*L1/(L0+amount)), amount + L1-(k/(L0+amount)))
    else:
        return ((k/(L1+amount)), L1+amount, amount + L0-(k/(L1+amount)))


def fpmm_trade_all(trades: List[Any], L0: float, L1: float, fee: float=0.02) -> (float, float):
    """Executes a cycle of trades on an fpmm."""
    for val, opt in trades:
        (L0, L1, rcv) = fpmm_trade(val, opt, L0, L1, fee)
        #print(L0, L1, rcv, L0*L1)
    return (L0, L1)


def simulate(nTrades0: int, nTrades1: int, amount: float, L0: float, L1: float, fee: float=0.02, iterations: int=1) -> float:
    """Simulates an fpmm."""
    p0 = 0.0
    trades0 = [(amount,0)]*nTrades0
    trades1 = [(amount,1)]*nTrades1
    trades = trades0 + trades1

    for _ in range(iterations):
        random.shuffle(trades)
        (L0_result, L1_result) = fpmm_trade_all(trades, L0, L1, fee)
        # The market probability of an option is proportional to its liquidity
        probability = L0_result / (L0_result + L1_result)
        p0 += probability
    
    return p0 / iterations


def test() -> None:
    trades = [(1.,0), (1.,0), (0.8,0), (0.8,0), (0.8,0), (0.32,0)]
    L0 = L1 = 10
    (L0, L1) = fpmm_trade_all(trades, L0, L1)
    p = L0/(L0+L1)
    expected_value = 0.6814355029609638
    tolerance = 1e-6
    assert abs(expected_value - p) < tolerance, "Market simulation do not match expected result."

test()

In [ ]:
# Trades per market
trades_per_market = [len(market_data.get('trades', {})) for market_data in fpmms.values()]
plot_histogram(trades_per_market, "Trades per market")

In [ ]:
# Collateral amounts invested
collateral_amounts = []
for market_data in fpmms.values():
    trades = market_data.get("trades", {})
    for trade_data in trades.values():
        collateral_amounts.append(float(trade_data["collateralAmount"])/1e18)

summary = describe(collateral_amounts)
print(summary)
limit = 2.0
print(len(collateral_amounts))
collateral_amounts = [x for x in collateral_amounts if x <= limit]
print(len(collateral_amounts))

plot_histogram(collateral_amounts, "Collateral amounts per bet (XDAI)")

In [ ]:
# Mech consistency
consistency = []
for market_data in fpmms.values():
    trades = market_data.get("trades", {})
    
    if trades:
        outcome_index_counts = Counter(trade_data["outcomeIndex"] for trade_data in trades.values())
        majority_outcome_index, majority_count = max(outcome_index_counts.items(), key=lambda x: x[1])
        consistency_percentage = (majority_count / len(trades)) * 100
        consistency.append(consistency_percentage)

plot_histogram(consistency, "Mech consistency")

In [ ]:
# Simulate markets

plt.ion()

# Create the figure and subplots
fig = plt.figure(figsize=(10, 5))
fig.subplots_adjust(wspace=0.4)
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122)
surf = None
ax1.set_xlabel('nTrades0')
ax1.set_ylabel('nTrades1')
ax1.set_zlabel('Market probability')
ax2.set_xlabel('nTrades')
ax2.set_ylabel('Market probability')
ax2.set_title('nTrades0 = nTrades * mech_consistency')

def update_plot(amount, liquidity, nTrades, mech_consistency, iterations, fee):
    global results, surf, surf_plane
    
    L0 = liquidity
    L1 = liquidity
    
    nTrades0 = int(nTrades * mech_consistency)
    nTrades1 = nTrades - nTrades0
    
    nTrades0_range = range(nTrades0+1)
    nTrades1_range = range(nTrades1+1)
    nTrades0_values, nTrades1_values = np.meshgrid(nTrades0_range, nTrades1_range)

    # Flatten the arrays
    nTrades0_values_flat = nTrades0_values.flatten()
    nTrades1_values_flat = nTrades1_values.flatten()
    
    results = np.array([simulate(nTrades0, nTrades1, amount, L0, L1, fee, iterations) for nTrades0, nTrades1 in zip(nTrades0_values_flat, nTrades1_values_flat)])
    results = results.reshape(nTrades0_values.shape)

    if surf is not None:
        surf.remove()
    surf = ax1.plot_surface(nTrades0_values, nTrades1_values, results, cmap='viridis')
    ax1.set_xlim(0, nTrades0)
    ax1.set_ylim(0, nTrades1)
    ax1.set_zlim(0, 1)
    
    if 'surf_plane' in globals():
        surf_plane.remove()
    surf_plane = ax1.plot_surface(nTrades0_values, nTrades1_values, np.full_like(results, mech_consistency), color='red', alpha=0.5)


    # Plot the diagonal of results
    results2 = np.array([results[int(t - int(t * mech_consistency)), int(t * mech_consistency)] for t in range(nTrades)])
    ax2.clear()
    ax2.plot(results2)
    ax2.axhline(y=mech_consistency, color='red', linestyle='--')
    ax2.set_xlim(0, nTrades)
    ax2.set_ylim(0, 1)


# Create a slider widget for amount
liquidity_slider = widgets.FloatSlider(value=10, min=0, max=10, step=0.1, description='Liquidity')
amount_slider = widgets.FloatSlider(value=1, min=0, max=10, step=0.01, description='Bet amount')
nTrades_slider = widgets.IntSlider(value=30, min=0, max=200, step=1, description='nTrades')
mech_consistency_slider = widgets.FloatSlider(value=0.75, min=0, max=1, step=0.01, description='Mech consistency')
iterations_slider = widgets.IntSlider(value=1, min=1, max=50, step=1, description='Iterations')
fee_slider = widgets.FloatSlider(value=0.02, min=0, max=1, step=0.01, description='Market fee')
widgets.interactive(update_plot, liquidity=liquidity_slider, amount=amount_slider, nTrades=nTrades_slider, mech_consistency=mech_consistency_slider, fee=fee_slider, iterations=iterations_slider)


Definitions:
- Market probability: Liquidity of the most voted option over the total liquidity. Ideally it should match the "population's belief" based on the trades.
- Market accuracy: The accuracy of the market in predicting the ground truth once resolved.
- Mech consistency: Percentage of the most voted option provided by the mech. This is the "population's belief".

Some observations:

- The market probability reflects what the traders think, which may or may not coincide with the actual result of the market once it is resolved. 
- The mech is abstracted as the collection of all the tools to which the traders interact.
- Asymptotically, i.e., with a large enough number of trades and large enough bet amounts, the market probability tends to the mech consistency. This is the best result we can hope for, since the market has no way to know "the ground truth" until it is resolved.
- This process can be "speed up" by either increasing the number of trades in the market, or by increasing the amount per trade, relative to the market liquidity.
- The simulation above does not distinguish between options (the problem is symmetrical).